In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
engine=create_engine("sqlite:///1.7 -funtional.db")#,echo=True)
Session=sessionmaker(bind=engine)
session=Session()

from sqlalchemy import MetaData, Table
metadata = MetaData()
alt_names=Table('Alt_names', metadata, autoload=True, autoload_with=engine)
inhibitor=Table('Inhibitor',metadata,autoload=True,autoload_with=engine)
kinase=Table('Kinase',metadata,autoload=True,autoload_with=engine)
kinase_inhibitor=Table('Kinase_Inhibitor',metadata,autoload=True,autoload_with=engine)
sub_phos=Table('Substrate_phosphosite',metadata,autoload=True,autoload_with=engine)
substrate_kinase=Table('Substrate_site_Kinase',metadata,autoload=True,autoload_with=engine)
substrate=Table('Substrate',metadata,autoload=True,autoload_with=engine)

C:\Users\44771\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['3']; using no arguments.
  (coltype, args))
C:\Users\44771\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['7']; using no arguments.
  (coltype, args))
C:\Users\44771\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['5']; using no arguments.
  (coltype, args))


In [26]:
filename="az20"
tsvfile=open(filename+".tsv","r")
newtsv=open(filename+"-kinase.csv","w")
line=tsvfile.readline()
line=line.split()
line.append("Kinase")
line.append("\n")
newtsv.write(",".join(line))

while True:
    line=tsvfile.readline()
    if line=="":
        break
    else:
        line=line.split()
        kn=line[0]
        kinlist=[]
        kn=kn.split("(")
        kn[1]=kn[1][:-1]
        subs=kn[0]
        modres=kn[1]
        try:
            kname=session.query(sub_phos).filter(sub_phos.c.Modified_residue==modres,sub_phos.c.Substrate_name==subs).one()
            kin=session.query(substrate_kinase).filter(substrate_kinase.c.Substrate_kinase_ID==kname.Substrate_kinase_ID).all()
            for x in kin:
                newtsv.write(",".join(line)+","+str(x.Kinase_name)+"\n")
                #print(line)

        except:
            try:
                sname=session.query(substrate).filter(substrate.c.Substrate_gene_name==subs).one()
                kname=session.query(sub_phos).filter(sub_phos.c.Modified_residue==modres,sub_phos.c.Substrate_name==sname.Substrate_name).one()
                kin=session.query(substrate_kinase).filter(substrate_kinase.c.Substrate_kinase_ID==kname.Substrate_kinase_ID).all()
                for x in kin:
                    newtsv.write(",".join(line)+","+str(x.Kinase_name)+"\n")
            except:
                pass
tsvfile.close()
newtsv.close()